In [1]:
!pip install mxnet

     |████████████████████████████████| 30.4 MB 4.9 MB/s eta 0:00:01


In [27]:
from mxnet import nd
from mxnet.gluon import nn
from mxnet.gluon.data import vision

In [34]:
### From the mxnet mnist tutorial
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import optimizer
from mxnet import autograd as ag

def mx_objective(config):
    # Fixing the random seed
    # mx.random.seed(42)
    mnist = mx.test_utils.get_mnist()
    train_data = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], config['batch_size'])
    val_data = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], config['batch_size'])

    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(128, activation='relu'))
        net.add(nn.Dense(64, activation='relu'))
        net.add(nn.Dense(10))

    gpus = mx.test_utils.list_gpus()
    ctx = [mx.gpu()] if gpus else [mx.cpu(0), mx.cpu(1)]
    net.initialize(mx.init.Uniform(scale=1), ctx=ctx)
    optim = optimizer.Adam(learning_rate=config['learning_rate'])
    trainer = gluon.Trainer(net.collect_params(), optim)

    epoch = config['epochs']
    # Use Accuracy as the evaluation metric.
    metric = mx.metric.Accuracy()
    softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
    for i in range(epoch):
    # Reset the train data iterator.
        train_data.reset()
        # Loop over the train data iterator.
        for batch in train_data:
        # Splits train data into multiple slices along batch_axis
        # and copy each slice into a context.
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        # Splits train labels into multiple slices along batch_axis
        # and copy each slice into a context.
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        # Inside training scope
        with ag.record():
            for x, y in zip(data, label):
                z = net(x)
                # Computes softmax cross entropy loss.
                loss = softmax_cross_entropy_loss(z, y)
                # Backpropagate the error for one iteration.
                loss.backward()
                outputs.append(z)
            # Updates internal evaluation
            metric.update(label, outputs)
            # Make one step of parameter update. Trainer needs to know the
            # batch size of data to normalize the gradient by 1/batch_size.
            trainer.step(batch.data[0].shape[0])
        # Gets the evaluation result.
        name, acc = metric.get()
        # Reset evaluation result to initial state.
        metric.reset()
        print('training acc at epoch %d: %s=%f' % (i, name, acc))

    # Use Accuracy as the evaluation metric.
    metric = mx.metric.Accuracy()
    # Reset the validation data iterator.
    val_data.reset()
    # Loop over the validation data iterator.
    for batch in val_data:
    # Splits validation data into multiple slices along batch_axis
    # and copy each slice into a context.
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
    # Splits validation label into multiple slices along batch_axis
    # and copy each slice into a context.
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        # Updates internal evaluation
        metric.update(label, outputs)
    print('validation acc: %s=%f' % metric.get())

In [37]:
config = {'learning_rate':.001, 'dropout':0.5, 'batch_size':64, 'epochs':50}

In [38]:
mx_objective(config)

training acc at epoch 0: accuracy=0.125000
training acc at epoch 1: accuracy=0.125000
training acc at epoch 2: accuracy=0.109375
training acc at epoch 3: accuracy=0.125000
training acc at epoch 4: accuracy=0.140625
training acc at epoch 5: accuracy=0.140625
training acc at epoch 6: accuracy=0.156250
training acc at epoch 7: accuracy=0.171875
training acc at epoch 8: accuracy=0.187500
training acc at epoch 9: accuracy=0.203125
training acc at epoch 10: accuracy=0.218750
training acc at epoch 11: accuracy=0.250000
training acc at epoch 12: accuracy=0.250000
training acc at epoch 13: accuracy=0.281250
training acc at epoch 14: accuracy=0.312500
training acc at epoch 15: accuracy=0.281250
training acc at epoch 16: accuracy=0.281250
training acc at epoch 17: accuracy=0.343750
training acc at epoch 18: accuracy=0.343750
training acc at epoch 19: accuracy=0.390625
training acc at epoch 20: accuracy=0.421875
training acc at epoch 21: accuracy=0.437500
training acc at epoch 22: accuracy=0.45312